# Setup

## Imports

In [14]:
import pandas as pd
import numpy as np
import os
import time
import random
import yfinance as yf
from datetime import datetime, timedelta
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix as cm

# Display all rows
pd.set_option('display.max_rows', None)

# Helper Functions

## get_random_startDates(n=40)

**Purpose:** This function generates a sequence of unique and random start dates. These dates fall within a specified range, starting from 59 days prior to the current date and ending yesterday
<br>
**Usage:** The primary application of these dates is to define the starting points for 1-day windows, each extending to the following day. This approach is particularly suited for acquiring stock data at 5-minute intervals within these 24-hour periods
<br>
**Features:** Avoids thanksgiving holidays and start dates that will create 1-day interval with a weekend, picks 35 dates since we will take 35 windows for each stock

In [15]:
def get_random_startDates(n=35):

    # Set range from 59 days before today till yesterday
    max_end_date = datetime.now().date() - timedelta(days=1)
    min_start_date = max_end_date - timedelta(days=56)

    # Avoid thanksgiving holiday
    excluded_dates = {
        datetime(2023, 11, 23).date(),
        datetime(2023, 11, 24).date()
    }

    # Generate a list of all valid dates
    valid_dates = [
        min_start_date + timedelta(days=i)
        for i in range((max_end_date - min_start_date).days + 1)
        if (min_start_date + timedelta(days=i) not in excluded_dates) and
        (min_start_date + timedelta(days=i)).weekday() < 5
    ]

    # Randomly select n dates from the list of valid dates
    if len(valid_dates) < n:
        raise ValueError(
            "Not enough valid dates available to meet the requested count.")

    start_dates = random.sample(valid_dates, n)
    start_dates = [date.strftime("%Y-%m-%d") for date in start_dates]

    return start_dates

## process_data(data, ticker, start_date, end_date, window_id)

**Purpose:** Perform basic data preprocessing to represent information like we want to see it

In [16]:
def process_data(data, ticker, start_date, end_date, window_id):

    # Rename upper case columns we are keeping
    data.rename(columns={ 'Open': 'open','Volume': 'volume' },inplace=True)

    # Create new columns for better representation
    data['stock'] = ticker
    data['start_date'] = f"{start_date}"
    data['window_ID'] = window_id
    data['timestep'] = range(1, len(data) + 1)
    data['volatility'] = data['open'] - data['Low'] 
    
    # Create target column
    # Shift the Close price to the previous row to compare it with the current open price
    data['target'] = (data['Close'].shift(-1) > data['open']).astype(int)


    # Rearrange columns to ensure consistent dataframe structure
    columns_order = [
        'stock', 'start_date', 'window_ID', 'timestep', 'open', 'volatility',
        'volume', 'target'
    ]
    return data[columns_order]

## shuffle_by_window(df)
**Purpose:** Shuffle the time-series interval for given df as per unique window_ID

In [17]:
def shuffle_by_window(df):
    
    # Get unique window_IDs
    window_ids = df['window_ID'].unique()
    
    # Shuffle the list of window_IDs
    np.random.shuffle(window_ids)
        
    # Concatenate all dataframes at once
    shuffled_df = pd.concat([df[df['window_ID'] == window_id] for window_id in window_ids], ignore_index=True)
    
    # Return the shuffled DataFrame
    return shuffled_df

## normalize_features(df, fts)
**Purpose:** Normalize the features given for the df provided

In [18]:
def normalize_features(df, fts):
    sc = StandardScaler()
    df[fts] = sc.fit_transform(df[fts])
    return df, sc

# Data Preprocessing

## Acquire Data

1. Apple - Highest Volume 50m approx - Technology
2. Delta Airlines - Above Average Volume 10m approx - Industrial
3. Capital One - Average Volume 3m approx - Finance
4. CVS - Medium Volume 1m approx - Healthcare

In [19]:
# Stocks used for training and testing the models
train_test_stocks = ['DIS','MTCH',
                     'TSLA','MHK',
                     'KO','GIS',
                     'EQT','KMI', 
                     'COF', 'PYPL',
                     'CVS', 'BSX',
                     'DAL','CSX',
                     'AAPL','FFIV',
                     'FMC','DOW',
                     'SPG','IRM',
                     'NI','XEL']

# Other unseen stocks to check model generalizability
unseen_stocks = ['NFLX','META','VZ','GOOGL',
                 'MCD','AMZN','NKE','POOL',
                 'DG','CLX','HSY','MO',
                 'COP','CVX','HAL','PSX',
                 'MS','MA','GPN','C',
                 'RMD','MRK','LLY','BAX',
                 'RSG','PWR','SNA','WAB',
                 'ZBRA','SNPS','ACN','MSFT',
                 'PKG','NEM','IFF','CF',
                'ARE','EQIX','HST','KIM',
                 'NRG','PCG','ATO','LNT']

# List of all stocks and shuffle them
stocks = train_test_stocks + unseen_stocks
random.shuffle(stocks)

# DataFrame to hold all data
all_data = pd.DataFrame()

# Window ID counter
window_id = 1

# Download and process the data for each stock and each interval
for stock in stocks:   
    
    # Print the stock being processed
    print(stock)
    
    # Take 35 random days for training stocks and 5 for rest
    days = 35 if stock in train_test_stocks else 5
    
    # Generate unique random start dates
    random_start_dates = get_random_startDates(n = days)
    
    # Loop through each data
    for start in random_start_dates:

        # Get end date for interval
        end = (datetime.strptime(start, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    
        # Download data for interval
        data = yf.download(tickers=stock, start=start, end=end, interval='5m',progress = False)
        
        # Process it to match our formatting
        processed_data = process_data(data, stock, start, end, window_id)
        
        # Add to the df holding all data
        all_data = pd.concat([all_data, processed_data], ignore_index=True)

        # Increment window ID
        window_id += 1  



# Drop the last row of each window where the shifted Close price would be NaN
all_data = all_data.groupby('window_ID').apply(lambda group: group.iloc[:-1]).reset_index(drop=True)

# Write readable all_data to CSV
all_data.to_csv(f"data/rd_all_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)


PKG
GOOGL
ARE
MHK
FFIV
C
MA
BSX
COF
BAX
RSG
TSLA
IRM
KO
NEM
VZ
MRK
PSX
ATO
CF
SPG
MTCH
COP
POOL
MS
DOW
RMD
MO
PYPL
NRG
DAL
SNPS
XEL
KIM
FMC
GPN
LNT
AAPL
HSY
MCD
CLX
NI
DG
EQT
CSX
DIS
NFLX
IFF
AMZN
KMI
HST
MSFT
PCG
EQIX
PWR
ZBRA
ACN
META
HAL
WAB
CVS
SNA
CVX
GIS
LLY
NKE


## Split train and test sets

In [20]:
# Separate dataframes to store training,testing and unseen data
train_data = pd.DataFrame()
test_data = pd.DataFrame()
unseen_data = pd.DataFrame()

# For each stock in train_test_stocks, the data goes to train_data or test_data based on the window_ID
for stock in train_test_stocks:
    stock_windows = all_data[all_data['stock'] == stock]['window_ID'].unique()
    stock_train = all_data[all_data['window_ID'].isin(stock_windows[:30])]
    stock_test = all_data[all_data['window_ID'].isin(stock_windows[30:35])]

    train_data = pd.concat([train_data, stock_train], ignore_index=True)
    test_data = pd.concat([test_data, stock_test], ignore_index=True)

    
# For each stock in unseen_stocks, the data goes to unseen_data
for stock in unseen_stocks:
    stock_unseen = all_data[all_data['stock'] == stock]
    unseen_data = pd.concat([unseen_data, stock_unseen], ignore_index=True)
    
    
# Shuffle train_data,test_data,unseen_data by window_ID
train_data = shuffle_by_window(train_data)
test_data = shuffle_by_window(test_data)
unseen_data = shuffle_by_window(unseen_data)

# Write readable train, test, unseen data to CSV before normalizing and dropping unnecessary columns
train_data.to_csv(f"data/rd_train_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)
test_data.to_csv(f"data/rd_test_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)
unseen_data.to_csv(f"data/rd_unseen_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)

,stock,start_date,window_ID,timestep,open,volatility,volume,target
0,DIS,2023-10-31,770,1,80.690002,0.279999,221818,0
1,DIS,2023-10-31,770,2,80.500000,0.080002,132538,1
2,DIS,2023-10-31,770,3,80.430000,0.090004,78192,0
3,DIS,2023-10-31,770,4,80.519997,0.169998,102693,1
4,DIS,2023-10-31,770,5,80.419998,0.089996,99674,1


## Normalize and Drop unrequired columns for ML

In [21]:
# Define fts to be normalized
fts = ['open', 'volatility', 'volume']

# Normalize features in training data
train_data, sc = normalize_features(train_data, fts)

# Apply the same transformation to the test,unseen data
test_data[fts] = sc.transform(test_data[fts])
unseen_data[fts] = sc.transform(unseen_data[fts])

# Drop columns stock, start_date since window_ID sufficient
train_data = train_data.iloc[:, 2:]
test_data = test_data.iloc[:, 2:]
unseen_data = unseen_data.iloc[:, 2:]

# Write train,test data to be actually used in ML to CSV
train_data.to_csv(f"data/ml_train_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)
test_data.to_csv(f"data/ml_test_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)
unseen_data.to_csv(f"data/ml_unseen_data_{datetime.now().strftime('%d-%m-%y')}.csv", index=False)

train_data.head()

,window_ID,timestep,open,volatility,volume,target
0,770,1,0.089041,1.200881,0.170300,0
1,770,2,0.085411,-0.032389,-0.051593,1
2,770,3,0.084073,0.029289,-0.186662,0
3,770,4,0.085793,0.522569,-0.125768,1
4,770,5,0.083882,0.029242,-0.133271,1


In [10]:
dt = yf.download(tickers=['AAPL','AMZN'], start='2023-10-14', end='2023-12-11', interval='5m',progress = False)

In [12]:
dt.head()

Adj Close                   Close              \
                                 AAPL        AMZN        AAPL        AMZN   
Datetime                                                                    
2023-10-16 09:30:00-04:00  178.129898  130.759995  178.129898  130.759995   
2023-10-16 09:35:00-04:00  177.869904  131.160004  177.869904  131.160004   
2023-10-16 09:40:00-04:00  177.300003  131.072693  177.300003  131.072693   
2023-10-16 09:45:00-04:00  177.110001  131.380005  177.110001  131.380005   
2023-10-16 09:50:00-04:00  177.009995  131.929993  177.009995  131.929993   

                                 High                     Low              \
                                 AAPL        AMZN        AAPL        AMZN   
Datetime                                                                    
2023-10-16 09:30:00-04:00  178.179993  131.070007  176.509995  130.425003   
2023-10-16 09:35:00-04:00  178.360001  131.440002  177.809998  130.750000   
2023-10-16 09:40:00-04:00  178.322495  131.630005  177.277496  130.970001   
2023-10-16 09:45:00-04:00  177.509995  131.719894  177.070007  131.000000   
2023-10-16 09:50:00-04:00  177.429993  131.970001  176.930099  131.384995   

                                 Open               Volume           
                                 AAPL        AMZN     AAPL     AMZN  
Datetime                                                             
2023-10-16 09:30:00-04:00  176.750000  130.690002  4407466  1974972  
2023-10-16 09:35:00-04:00  178.119995  130.835007  1454933  1003305  
2023-10-16 09:40:00-04:00  177.869995  131.172607  1141014   837142  
2023-10-16 09:45:00-04:00  177.320007  131.085007   882999   728300  
2023-10-16 09:50:00-04:00  177.119995  131.389999   774887   746733